# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You should use this notebook as a reference/documenation for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange. If everything is setup correctly, you should see the line "Setup was successful." being printed.

In [1]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


### Define Your Instrument

An instrument is the term we use for the 'thing' that we are trading. As such, if we are trading BMW stocks, we would call those an instrument. However, BMW options would be a different instrument. 

This field determines which instrument we are trading. By changing it, you will insert trades for a different instrument. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [2]:
instrument_id = 'AAPL'

### Connect to Exchange

In [3]:
#######################################
# FILL IN OWN NAME AND PASSWORD BELOW #
#######################################

e = Exchange(host='chi-tech-starters.optibook.net')
a = e.connect(username='PerfectHedge', password='2v3gy923nr')

2020-08-14 08:42:04,588 [asyncio   ] [MainThread  ] Using selector: KqueueSelector


## Outstanding Orders, Trades, Current Positions and PnL

In [ ]:
# Returns all currently outstanding orders
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o)

In [ ]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)
for t in trades:
    print(t)

In [ ]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(t)

In [ ]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

In [ ]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

In [ ]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

## Order Book and Public Trade Ticks

In [ ]:
book = e.get_last_price_book(instrument_id)
print(book)

In [ ]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(t)

In [ ]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)

In [ ]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

## Inserting and Deleting Orders

In [15]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use to buy.
result = e.insert_order(instrument_id, price=450.0, volume=1, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 4


In [14]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use to sell.
result = e.insert_order(instrument_id, price=450.0, volume=8, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 3


In [18]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
# Use to buy.
result = e.insert_order(instrument_id, price=450.0, volume=1, side='bid', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 6


In [ ]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
# Use to sell.
result = e.insert_order(instrument_id, price=122.0, volume=1, side='ask', order_type='ioc')
print(f"Order Id: {result}")

In [ ]:
# Attempt to delete inserted order by order_id
order_id = 4
result = e.delete_order(instrument_id, order_id=order_id)
print(f"Deleted order id {order_id}: {result}")

In [ ]:
# Change volume for existing order
order_id = 0
new_volume = 10
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}")

In [4]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

## 'Hack' Out of Positions

In [5]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'ALLIANZ': 0, 'SAP': 0, 'UNILEVER': -117, 'ASML': 0, 'AIRBUS': 0, 'LVMH': 0, 'SIEMENS': 0, 'TOTAL': 45}
{'ALLIANZ': 0, 'SAP': 0, 'UNILEVER': 0, 'ASML': 0, 'AIRBUS': 0, 'LVMH': 0, 'SIEMENS': 0, 'TOTAL': 0}
